<a href="https://colab.research.google.com/github/HanifFaishalH/pbl_jawara/blob/main/Model_PCVK_Kel3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Mengambil Dataset Batik

In [17]:
import pandas as pd
import os

# path folder dataset
source_data = '/content/drive/MyDrive/PBL/batik-dataset'

classes = os.listdir(source_data)

data_info = []

for kelas in classes:
    kelas_path = os.path.join(source_data, kelas)

    # Hitung jumlah file gambar
    jumlah_file = len([f for f in os.listdir(kelas_path)
                       if os.path.isfile(os.path.join(kelas_path, f))])

    data_info.append({
        "Kategori": kelas,
        "Jumlah Gambar": jumlah_file
    })

# Buat DataFrame informasi dataset
df = pd.DataFrame(data_info)

print(df)

  Kategori  Jumlah Gambar
0    Tulis             48
1      Cap             48


# Proses Augmentasi Dataset
- Horizonal Flip
- Vertikal Flip
- Zoom 50%

In [18]:
import os
import shutil
from PIL import Image
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm

source_data = '/content/drive/MyDrive/PBL/batik-dataset'
base_augmented_output = '/content/drive/MyDrive/PBL/batik-dataset-augmented'
ORIGINAL_IMAGES_PER_CLASS = 48

AUGMENTATION_CONFIGS = [
    {"name": "zoom", "datagen_params": {"zoom_range": [0.5, 1.0], "fill_mode": 'nearest'}},
    {"name": "vflip", "datagen_params": {"vertical_flip": True, "fill_mode": 'nearest'}},
    {"name": "hflip", "datagen_params": {"horizontal_flip": True, "fill_mode": 'nearest'}}
]

if not os.path.exists(base_augmented_output):
    os.makedirs(base_augmented_output)

print(f"Memulai proses augmentasi dari: {source_data}")

classes = os.listdir(source_data)
total_overall_final = 0

# Loop utama untuk setiap kelas
for class_name in classes:
    source_class_path = os.path.join(source_data, class_name)
    target_class_path = os.path.join(base_augmented_output, class_name)

    if not os.path.exists(target_class_path):
        os.makedirs(target_class_path)

    # Filter hanya file gambar
    image_files = [f for f in os.listdir(source_class_path)
                   if os.path.isfile(os.path.join(source_class_path, f))
                   and f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    if not image_files:
        print(f"-> Melewati kelas '{class_name}': Tidak ditemukan gambar.")
        continue

    original_count = 0
    augmented_total_count = 0

    print(f"\nMemproses Kelas '{class_name}' ({len(image_files)} gambar asli)")

    for original_image_name in tqdm(image_files, desc=f"Augmentasi {class_name}"):
        original_image_path = os.path.join(source_class_path, original_image_name)

        # 1. SALIN FILE ASLI
        try:
            shutil.copy(original_image_path, os.path.join(target_class_path, original_image_name))
            original_count += 1
        except Exception as e:
            print(f"   - Gagal menyalin gambar asli '{original_image_name}'. Error: {e}")
            continue

        # 2. PROSES AUGMENTASI
        try:
            img = Image.open(original_image_path)
            x = np.asarray(img, dtype='float32')
            if x.ndim == 2:
                x = np.expand_dims(x, axis=-1)
            x = x.reshape((1,) + x.shape)

            for aug_config in AUGMENTATION_CONFIGS:
                aug_name = aug_config["name"]
                aug_params = aug_config["datagen_params"]

                datagen_discrete = ImageDataGenerator(**aug_params)

                for batch in datagen_discrete.flow(x, batch_size=1,
                                                 save_to_dir=target_class_path,
                                                 save_prefix=f'{aug_name}_{original_image_name.split(".")[0]}',
                                                 save_format='jpeg'):
                    augmented_total_count += 1
                    break

        except Exception as e:
            print(f"   - Gagal mengaugmentasi gambar '{original_image_name}'. Error: {e}")

    total_final = original_count + augmented_total_count
    total_overall_final += total_final

    print(f"Kelas '{class_name}' selesai diproses.")
    print(f"Total Gambar Asli (disalin ke folder akhir): {original_count}")
    print(f"Total Augmentasi yang Dibuat: {augmented_total_count} ({original_count} x 3)")
    print(f"TOTAL GAMBAR AKHIR PER KELAS: {total_final}")
    print("-" * 50)


print("\nProses augmentasi diskrit selesai!")
expected_per_class = ORIGINAL_IMAGES_PER_CLASS + (ORIGINAL_IMAGES_PER_CLASS * len(AUGMENTATION_CONFIGS))
print(f"SETIAP KELAS SEKARANG MENGANDUNG TOTAL {ORIGINAL_IMAGES_PER_CLASS} ASLI + {ORIGINAL_IMAGES_PER_CLASS * len(AUGMENTATION_CONFIGS)} AUGMENTED = {expected_per_class} GAMBAR.")

Memulai proses augmentasi dari: /content/drive/MyDrive/PBL/batik-dataset

Memproses Kelas 'Tulis' (48 gambar asli)


Augmentasi Tulis: 100%|██████████| 48/48 [05:09<00:00,  6.46s/it]


Kelas 'Tulis' selesai diproses.
Total Gambar Asli (disalin ke folder akhir): 48
Total Augmentasi yang Dibuat: 144 (48 x 3)
TOTAL GAMBAR AKHIR PER KELAS: 192
--------------------------------------------------

Memproses Kelas 'Cap' (48 gambar asli)


Augmentasi Cap: 100%|██████████| 48/48 [04:54<00:00,  6.13s/it]

Kelas 'Cap' selesai diproses.
Total Gambar Asli (disalin ke folder akhir): 48
Total Augmentasi yang Dibuat: 144 (48 x 3)
TOTAL GAMBAR AKHIR PER KELAS: 192
--------------------------------------------------

Proses augmentasi diskrit selesai!
SETIAP KELAS SEKARANG MENGANDUNG TOTAL 48 ASLI + 144 AUGMENTED = 192 GAMBAR.
